### Trying out Bagging

We'll use the pima indian diabetes dataset to predict if a person has diabetes or not based on certain features such as blood pressure, insulin, bmi, age, etc. First we'll try a standalone model and then use bagging (ensemble technique) to check how we can improve the performance of the model.

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv("pima_indians_diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [15]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Looking at the descriptive stats, we can see that the features are not on a similar scale, we'll use standard scaler to fix this.

In [50]:
df.Outcome.value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

#### Creating our feature matrix

In [51]:
X = df.iloc[:, :-1]
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


#### The label vector

In [18]:
y = df.Outcome
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

#### Feature Scaling

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().set_output(transform="pandas")
X_scaled = scaler.fit_transform(X)
X_scaled.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


In [24]:
X_scaled.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,-6.476301e-17,-9.251859e-18,1.503427e-17,1.006140e-16,-3.006854e-17,2.590520e-16,2.451743e-16,1.931325e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-3.783654e+00,-3.572597e+00,-1.288212e+00,-6.928906e-01,-4.060474e+00,-1.189553e+00,-1.041549e+00
25%,-8.448851e-01,-6.852363e-01,-3.673367e-01,-1.288212e+00,-6.928906e-01,-5.955785e-01,-6.889685e-01,-7.862862e-01
50%,-2.509521e-01,-1.218877e-01,1.496408e-01,1.545332e-01,-4.280622e-01,9.419788e-04,-3.001282e-01,-3.608474e-01
75%,6.399473e-01,6.057709e-01,5.632228e-01,7.190857e-01,4.120079e-01,5.847705e-01,4.662269e-01,6.602056e-01
max,3.906578e+00,2.444478e+00,2.734528e+00,4.921866e+00,6.652839e+00,4.455807e+00,5.883565e+00,4.063716e+00


The features are now on a more similar scale.

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [34]:
X_train.shape, y_train.shape

((614, 8), (614,))

In [35]:
X_test.shape, y_test.shape

((154, 8), (154,))

In [36]:
y_train.value_counts()

Outcome
0    401
1    213
Name: count, dtype: int64

#### Training a simple decision tree model

Lets just try to fit a simple Decision tree on our data and see how it performs

In [53]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), X_scaled, y, cv=5)
print("The scores on the different folds are: ", scores)
print("The mean score is: ", scores.mean())

The scores on the different folds are:  [0.68181818 0.68831169 0.69480519 0.77124183 0.73856209]
The mean score is:  0.7149477973007384


#### Trying out BaggingClassifier

Let's try a bagging classifier that trains 50 decision trees and uses them to make the predictions

In [59]:
from sklearn.ensemble import BaggingClassifier

model1 = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=50,
    random_state=42
)

scores = cross_val_score(model1, X_scaled, y, cv=5)
print("The scores on the different folds are: ", scores)
print("The mean score is: ", scores.mean())

The scores on the different folds are:  [0.74675325 0.71428571 0.76623377 0.83006536 0.76470588]
The mean score is:  0.7644087938205586


We can clearly see that accuracy of the model increased when using the bagging classifier trained on 50 trees compared to the old standalone decision tree.
Using a Bagging classifier with decision tree as the estimator is similar to using a RandomForestClassifer. We can see that they perform very similar:

In [66]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(
    RandomForestClassifier(n_estimators=50),
    X=X_scaled,
    y=y,
    cv=5
)

print("The scores on the different folds are: ", scores)
print("The mean score is: ", scores.mean())

The scores on the different folds are:  [0.75324675 0.72077922 0.75974026 0.83006536 0.76470588]
The mean score is:  0.7657074951192598


We can also use different estimators in our bagging classifier other than decision trees:

In [63]:
from sklearn.linear_model import LogisticRegression

model2 = BaggingClassifier(
    estimator=LogisticRegression(),
    n_estimators=50,
    random_state=42
)

scores = cross_val_score(model2, X_scaled, y, cv=5)
print("The scores on the different folds are: ", scores)
print("The mean score is: ", scores.mean())

The scores on the different folds are:  [0.77272727 0.74675325 0.74675325 0.81699346 0.75816993]
The mean score is:  0.7682794329853152


In [64]:
from sklearn.neighbors import KNeighborsClassifier

model3 = BaggingClassifier(
    estimator=KNeighborsClassifier(),
    n_estimators=50,
    random_state=42
)

scores = cross_val_score(model3, X_scaled, y, cv=5)
print("The scores on the different folds are: ", scores)
print("The mean score is: ", scores.mean())

The scores on the different folds are:  [0.72727273 0.73376623 0.75974026 0.78431373 0.7124183 ]
The mean score is:  0.7435022493846024


In [65]:
from sklearn.svm import SVC

model4 = BaggingClassifier(
    estimator=SVC(),
    n_estimators=50,
    random_state=42
)

scores = cross_val_score(model4, X_scaled, y, cv=5)
print("The scores on the different folds are: ", scores)
print("The mean score is: ", scores.mean())

The scores on the different folds are:  [0.75974026 0.75974026 0.75974026 0.82352941 0.77124183]
The mean score is:  0.7747984042101688
